# Data Pull

In [9]:
pip install ccxt

In [10]:
import ccxt
import time
import pandas as pd
from datetime import datetime, timezone
from tqdm import tqdm
import os

## Coin Universe

In [18]:
SPOT_COLUMNS = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume',
                'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

binance_exchange = ccxt.binanceus({
    'timeout': 15000,
    'enableRateLimit': True
    # 'options': {'defaultType': 'future'}
})
markets = binance_exchange.load_markets()
all_tickers = list(markets.keys())
usdt_tickers = [ticker for ticker in all_tickers if ticker.endswith('USDT')]
active_usdt_tickers = [
    ticker for ticker in usdt_tickers if markets[ticker]['active']
]
len(active_usdt_tickers)

162

In [19]:
def get_spot(exchange, symbol, interval = '1d',
                    startTime = None,
                    endTime = None,
                    limit = 1000):

    if (startTime == None and endTime == None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'limit': limit})
    elif (startTime == None and endTime != None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'endTime': endTime,
                                        'limit': limit})
    elif (startTime != None and endTime == None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'startTime': startTime,
                                        'limit': limit})
    else:
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'limit': limit})

def convert_to_seconds(time_input):
    number = int(time_input[:-1])
    unit = time_input[-1]

    if unit == 's':
        return number
    elif unit == 'm':
        return number * 60
    elif unit == 'h':
        return number * 3600
    elif unit == 'd':
        return number * 86400
    else:
        raise ValueError("Unsupported time unit")

def transform_timestamp(timestamp_integer):
    '''
    As data points involved milliseconds, we need to transform them by constant 1000.
    '''

    return pd.to_datetime(int(timestamp_integer / 1000), utc=True, unit='s')

def transform_to_timestamp_integer(datetime_object):
    '''
    As data points involved milliseconds, we need to transform them by constant 1000.
    '''

    return int(datetime_object.timestamp() * 1000)

def obtain_full_spotdata(start_timestamp,
                         end_timestamp,
                         exchange, symbol, interval = '1h',
                         limit = 1000):

    time_difference = int(convert_to_seconds(interval) * limit * 1000)

    full_data_list = []

    curr_time = start_timestamp + time_difference
    while (curr_time + time_difference < end_timestamp):
        data_list = get_spot(exchange = exchange, symbol = symbol, interval = interval,
                             endTime = curr_time,
                             limit = limit)
        full_data_list = full_data_list + data_list

        time.sleep(0.2)
        curr_time += time_difference

    data_list = get_spot(exchange = exchange, symbol = symbol, interval = interval,
                        startTime = curr_time,
                        endTime = end_timestamp,
                        limit = limit)

    full_data_list = full_data_list + data_list

    return full_data_list

In [20]:
start_date = "2016-01-01"
end_date = "2025-02-24"
interval = "1d"

In [21]:
def fetch_data(ticker_symbol, start_date, end_date, interval):
    # Convert dates to timestamps in milliseconds
    start_timestamp = int(((pd.to_datetime(start_date)).tz_localize('UTC')).timestamp() * 1000)
    end_timestamp = int(((pd.to_datetime(end_date)).tz_localize('UTC')).timestamp() * 1000)

    # Obtain raw data from Binance US
    full_data_list = obtain_full_spotdata(start_timestamp, end_timestamp, binance_exchange, ticker_symbol, interval=interval)

    # Create a DataFrame with the correct column names
    data = pd.DataFrame(full_data_list, columns=SPOT_COLUMNS)
    data['Open time'] = data['Open time'].apply(lambda x: transform_timestamp(int(x)))

    # Remove any duplicate entries based on 'Open time'
    data.drop_duplicates('Open time', keep='first', inplace=True)

    # Add a column to identify the ticker
    data['Ticker'] = ticker_symbol
    return data

In [22]:
all_data = []
for ticker in tqdm(active_usdt_tickers, desc="Fetching crypto data"):
    try:
        cleaned_ticker = ticker.replace('/', '')
        df = fetch_data(cleaned_ticker, start_date, end_date, interval)
        all_data.append(df)
        print(f"Data for {cleaned_ticker} fetched successfully.")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Concatenate all the DataFrames into one master DataFrame
master_data = pd.concat(all_data, ignore_index=True)

# Save the master DataFrame to a Parquet file
master_data.to_parquet("master_crypto_data.parquet", index=False)
print("Master parquet file saved as 'master_crypto_data.parquet'")

Fetching crypto data:   1%|          | 1/162 [00:00<02:25,  1.11it/s]

Data for BTCUSDT fetched successfully.


Fetching crypto data:   1%|          | 2/162 [00:01<02:13,  1.20it/s]

Data for ETHUSDT fetched successfully.


Fetching crypto data:   2%|▏         | 3/162 [00:02<02:07,  1.25it/s]

...

Fetching crypto data:  99%|█████████▉| 161/162 [01:54<00:00,  1.53it/s]

Data for TRUMPUSDT fetched successfully.


Fetching crypto data: 100%|██████████| 162/162 [01:54<00:00,  1.41it/s]

Data for NEIROUSDT fetched successfully.


Master parquet file saved as 'master_crypto_data.parquet'
